In [90]:
import os, sys
sys.path.insert(0, '/home/jtorrenc/mvesc/ETL')
from mvesc_utility_functions import *
import numpy as np
import pandas as pd
import random
from functools import partial

In [286]:
def color_gt(val,x=.5,c='blue',neg=True):
    """
    Colors values greater than x blue (or other provided color)
    
    :param float x: lower bound on colored values
    :param str c: color
    :param bool neg: if true, color negative values red
    :returns: css string to color values in dataframe
    :rtype: str
    """
    
    if val > x:
        color = c 
    elif val < 0:
        color = 'red' 
    else: color = 'black'
    return 'color: %s' % color

def table_exists(cursor, table, schema='clean'):
    """
    Checks to see if a table exists in the database
    
    :param pg.cursor cursor: 
    :param str table:
    :param str schema:
    :rtype: bool
    """
    cursor.execute("""                                                   
            select count(*) from information_schema.tables                       
            where table_schema = %s and table_name = %s
            """, [schema, table])
    return cursor.fetchall()[0][0]

In [129]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        best = dict()
        for m in model_list:
            cursor.execute("""select filename from model.reports where model_name = %s
                            order by val_precision_10 desc limit 1; """, [m])
            best[m] = cursor.fetchall()[0][0]
    predictions = pd.DataFrame()
    for m, f in best.items():
        df = read_table_to_df(connection, f,columns = ['student_lookup','predicted_score'],
                                          schema='predictions')
        df.set_index('student_lookup',inplace=True)
        df.columns = [m]
        predictions = pd.concat((predictions,df), axis=1, join='inner')


In [269]:
kendall_pred = predictions.corr('kendall')
spearman_pred = predictions.corr('spearman')

# Kendall's Tau Prediction Correlation

In [272]:
kendall_pred.style.applymap(partial(color_gt,x=.65))

# Spearman's Prediction Correlation

In [277]:
spearman_pred.style.applymap(partial(color_gt,x=.75))

In [229]:
with postgres_pgconnection_generator() as connection:
    with connection.cursor() as cursor:
        cursor.execute('select distinct model_name from model.reports')
        model_list = [m[0] for m in cursor.fetchall()]
        cursor.execute('select distinct feature_categories from model.reports')
        feature_list = [f[0] for f in cursor.fetchall()]
        to_remove = []
        for f in feature_list:
            if ',' in f:
                to_remove.append(f)
        for f in to_remove:
            feature_list.remove(f)
        best = dict()
        features = dict()
        spearman = dict()
        for f in feature_list:
            best[f] = dict()
            for m in model_list:
                cursor.execute("""select filename from model.reports 
                                where model_name = %s
                                    and feature_categories = %s
                                    and feature_grades = '5, 6, 7, 8, 9'
                                order by val_precision_10 desc limit 1; """, [m,f])
                best[f][m] = cursor.fetchall()[0][0]
            temp = pd.DataFrame()
            for m, file in best[f].items():
                if table_exists(cursor, file, 'feature_scores'):
                    df = read_table_to_df(connection, file ,columns=['feature','importance'],schema='feature_scores',nrows = -1)
                    df.set_index('feature',inplace=True)
                    df.columns = [m]
                    temp = pd.concat((temp,df), axis=1, join='outer')
            features[f] = temp
            spearman[f] = temp.corr('spearman')

In [252]:
best['mobility']

{'AB': 'param_set_453_AB_ht_4524',
 'DT': 'param_set_458_DT_ht_4571',
 'ET': 'param_set_453_ET_ht_4523',
 'GB': 'param_set_468_GB_ht_4676',
 'KNN': 'param_set_458_KNN_ht_4579',
 'NB': 'param_set_463_NB_ht_4627',
 'RF': 'param_set_463_RF_ht_4622',
 'SGD': 'param_set_453_SGD_ht_4528',
 'SVM': 'param_set_453_SVM_ht_4525',
 'logit': 'param_set_453_logit_ht_4520'}

# Spearman's Feature Correlation


In [253]:
features['absence'].head()

,GB,AB,RF,DT,SVM,SGD,ET,logit
absence_consec_gr_7,0.000000,0.000000,0.000789,0.000000,0.087852,22.639499,0.000924,0.262957
absence_consec_gr_8,0.011905,0.003216,0.028656,0.000000,-0.000016,1.331735,0.028532,0.031852
absence_consec_gr_9,0.060845,0.099215,0.046819,0.040512,0.000055,9.322147,0.046311,0.269176
absence_gr_5,0.078696,NaN,0.064056,0.020813,-0.000023,-8.101389,0.073802,-0.046104
absence_gr_6,0.081279,0.052965,0.079172,0.127702,0.000008,-9.943623,0.079043,-0.222509


In [287]:
spearman['absence'].style.applymap(partial(color_gt,x=.8))

In [283]:
features['mobility'].head()


,GB,AB,RF,DT,SVM,SGD,ET,logit
avg_address_change_to_gr_5,0.006284,0.0,0.002644,0.000000,0.418592,1.331735e+00,0.001653,0.000000
avg_address_change_to_gr_5_isnull,0.000010,0.0,0.004189,0.007397,0.280750,-1.512494e-14,0.002890,0.041814
avg_address_change_to_gr_6,0.000757,0.0,0.005937,0.000000,0.071333,5.553688e-15,0.004239,0.000000
avg_address_change_to_gr_6_isnull,0.000000,0.0,0.002541,0.000000,-0.000069,-1.276167e-14,0.002361,0.005640
avg_address_change_to_gr_7,0.008888,0.0,0.004179,0.000000,-0.014458,-2.219559e-01,0.003427,0.000000


In [288]:
spearman['mobility'].style.applymap(partial(color_gt,x=.8))

In [257]:
features['oaa_normalized'].head()

,GB,AB,RF,DT,SVM,SGD,ET,logit
eighth_math_normalized,0.038644,0.127352,0.039856,0.000000,0.085535,2.138664,0.021150,0.000000
eighth_math_normalized_isnull,0.000000,0.000000,0.006814,0.000000,0.169345,5.326941,0.004572,0.000000
eighth_math_percentile,0.018720,0.044069,0.034348,0.080650,-0.055547,-2.593598,0.035541,0.000000
eighth_math_percentile_isnull,0.004537,0.000000,0.006687,0.025544,0.169345,5.326941,NaN,0.000000
eighth_math_pl_Accelerated,0.000000,0.000000,0.002173,0.000000,-0.008385,-2.663471,0.005935,-0.233912


In [289]:
spearman['oaa_normalized'].style.applymap(partial(color_gt,x=.8))

In [259]:
features['snapshots'].head()


,GB,AB,RF,DT,SVM,SGD,ET,logit
days_absent_excused_gr_7,0.014806,0.023076,0.026768,0.000000,-0.713126,-156.283049,0.019405,-0.505254
days_absent_excused_gr_8,0.010918,0.000000,0.023392,0.000000,0.100783,17.032193,0.022353,0.000000
days_absent_excused_gr_9,0.024187,0.000000,0.069043,0.032535,0.394523,-0.003329,0.057544,0.351827
days_absent_gr_5,0.022970,0.000000,0.020311,0.017877,-0.063759,-159.031385,0.016444,-0.061790
days_absent_gr_6,0.017842,0.014429,0.019935,0.000000,-0.036667,-145.958184,0.016859,-0.096762


In [290]:
spearman['snapshots'].style.applymap(partial(color_gt,x=.7))

In [291]:
pd.Panel(spearman).mean(axis=0).style.applymap(partial(color_gt,x=.8,c='blue'))